# Kartendarstellung

In [ ]:
import pandas as pd
from src.user_lib.read_file import *

#from tqdm import tqdm

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import folium

In [ ]:
# pathtofile = 'data/Rechnungen_Stadtmobil.xlsx'
# sheetname = 'Rechnungen_Stadtmobil'
# df = readDataFromXls(pathtofile,sheetname)
# df.to_pickle('data/df.pkl')
df= pd.read_pickle('data/df.pkl')

df.sample(5)

In [ ]:

def cleanaddress(str):
    try:
        str=str.split(',')[1]
    except:
        str=str 
    return str


In [ ]:

#df['Adresse'] = df.apply(lambda x: cleanaddress(x['Adresse']), axis=1)
#df[df['Adresse'].str.contains('c/o')==True]

## get coordinates for addresses from OSM-API

In [ ]:
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="test_agent")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#location = geolocator.geocode("175 5th Avenue NYC")
location = geolocator.geocode("Johann-Weiß-Str. 18 68167 Mannheim")

print(location.address)
#Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
print((location.latitude, location.longitude))
#(40.7410861, -73.9896297241625)
print(location.raw)
#{'place_id': '9167009604', 'type': 'attraction', ...}


In [ ]:
# nur die daten mit 
df_buchungen = df[df['km-/Kraftstoffpreis'].isna()==False]
df_buchungen.sample(3)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test_agent")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


#tqdm.pandas()
#df_mitglieder['location'] = df_mitglieder['Anschrift'].progress_apply(geocode)
df_buchungen['location'] = df_buchungen['Adresse'].apply(geocode)
df_buchungen['point'] = df_buchungen['location'].apply(lambda loc: tuple(loc.point) if loc else None)

## Konvertierung in Geodataframe

In [ ]:
df_buchungen['geometry'] = df_buchungen['location'].apply(lambda loc: (loc.longitude, loc.latitude) if loc else None)
df_buchungen.tail(2)
#coordinate_values = (location.longitude, location.latitude)  #in geopandas, the x value corresponds to the longitude while the y value, the latitude(Just in case you were wondering why it was *location.longitude, location.latitude* and not the other way round )

In [ ]:
# check, if there are any locations empty ==> shall be removed for further processing
df_buchungen[df_buchungen['geometry'].isna()] 


In [ ]:
df_buchungen=df_buchungen[~df_buchungen['geometry'].isna()]

In [ ]:
print(df_buchungen['geometry'].head())

In [ ]:
df_buchungen['geometry']

In [ ]:
df_buchungen['geometry'] = df_buchungen['geometry'].apply(Point)
gdf = gpd.GeoDataFrame(df_buchungen, geometry='geometry')#, crs='epsg:4326')
gdf.head(2)


In [ ]:
pd.to_pickle(gdf,"./data/gdf.pkl")

In [ ]:
gdf.columns

In [ ]:
#gdf = gdf.astype(str)
gdf=pd.read_pickle("./data/gdf.pkl")
gdf.drop(['Dauer','Dauer_hours', 'Buchungsnr', 'location', 'point'], axis="columns", inplace=True)

gdf=gdf.replace(to_replace=[r'ä', r'ö', r'ü', r'Ä', r'Ö', r'Ü', r'ß'],
              value=['ae', 'oe', 'ue','Ae', 'Oe', 'Ue', 'ss'],regex=True) 

In [ ]:
gdf.head()

In [ ]:
#pd.to_pickle(gdf,"./data/gdf_mitglieder.pkl")
gdf.crs = 'epsg:4326'
gdf.to_file('./data/locations.geojson', driver="GeoJSON")  

In [ ]:
gdf.head(2)

In [ ]:
gjson = gdf['geometry'].to_json()


In [ ]:


m = folium.Map(location=[49.50786825, 8.489917597582128], tiles='cartodb positron', zoom_start=14)
locs = folium.features.GeoJson(gjson)
m.add_child(locs)
folium.Marker(location=[49.50786825, 8.489917597582128],popup = 'home',
                  icon= folium.Icon(color='green',
                  icon_color='yellow',icon = 'home')).add_to(m)
m  


In [ ]:
statusdict={'Diesel':'brown',
        'Benzin':'red',
        'Elektro':'lightgreen',
        # 'O':'orange',
        # 'M':'pink',
        # 'R':'red',
        # 'HG':'lightgreen',
        # 'RS':'lightred',
        # 'B':'blue'
        }
#[‘red’, ‘blue’, ‘green’, ‘purple’, ‘orange’, ‘darkred’,
#’lightred’, ‘beige’, ‘darkblue’, ‘darkgreen’, ‘cadetblue’, ‘darkpurple’, ‘white’, ‘pink’, ‘lightblue’, ‘lightgreen’, ‘gray’, ‘black’, ‘lightgray’]

def setstatusmitgliedschaftfarbe(dict,status):
    color = 'lightgray'
    try:
        color = dict[status]
    except:
        color = 'lightgray'
    return color


In [ ]:
m = folium.Map(location=[52.47468939414727, 13.416986828692476], zoom_start=20, max_zoom=30, 
               tiles="OpenStreetMap", overlay=True, control=False)
tooltip = "Click me!"
layer1 = folium.FeatureGroup(name='Anschriften', overlay=True, control=True)    
for _, r in gdf.iterrows():
    lat = r['point'].y
    lon = r['point'].x
    #popupcolor,popupicon,popuptext=setpopupstyle(r['Art'],r['BMK_Loc'])
    popupcolor=setstatusmitgliedschaftfarbe(dict,r['Kraftstoff'])
    folium.Marker(location=[lat, lon],
                  #popup=r['Vor+Nachname'],
                  icon=folium.Icon(color=popupcolor) #, icon=popupicon
                  ).add_to(layer1)
layer1.add_to(m)

#layer2 = folium.FeatureGroup(name='Vereine', overlay=True, control=True)       
#for _, r in dfGrundriss.iterrows():
##    # Without simplifying the representation of each borough,
##    # the map might not be displayed
#    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
#    geo_j = sim_geo.to_json()
#    geo_j = folium.GeoJson(data=geo_j,
#                           style_function=lambda x: {'fillColor': 'orange'},popup=r['Verein'])
#    folium.Popup(r['Verein']).add_to(geo_j)
#    geo_j.add_to(layer2)
#layer2.add_to(m)   

#layer3 = folium.FeatureGroup(name='Farbgruppen', overlay=True, control=True)      
#for _, r in dfFarbgruppen.iterrows():
##    # Without simplifying the representation of each borough,
##    # the map might not be displayed
#    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
#    geo_j = sim_geo.to_json()
#    geo_j = folium.GeoJson(data=geo_j,
#                           style_function=lambda x: {'fillColor': setfarbgruppenfarbe(farbendict,r['Farbe'])},popup=r['Farbgruppe'])
#    folium.Popup(r['Farbgruppe']).add_to(geo_j)
#    geo_j.add_to(layer3)
#layer3.add_to(m)    

#layer4 = folium.FeatureGroup(name='Beete Umriss', overlay=True, control=True)     
#for _, r in dfBeete.iterrows():
##    # Without simplifying the representation of each borough,
##    # the map might not be displayed
#    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
#    geo_j = sim_geo.to_json()
#    #color=setpopupstyle(r['StatusRückmeldung'])
#    text='Beetname: {} <br> StatusRückmeldung: {}'.format(r['Beetname'], r['StatusRückmeldung'])
#    geo_j = folium.GeoJson(data=geo_j,
#                           style_function=lambda x: {'fillColor': setpopupstyle(r['StatusRückmeldung'])})
#    folium.Popup(text).add_to(geo_j)
#    geo_j.add_to(layer4)
#    layer4.add_to(m)    
##for _, r in df.iterrows():
##    lat = r['centroid'].y
##    lon = r['centroid'].x
##    #popupcolor,popupicon,popuptext=setpopupstyle(r['Art'],r['BMK_Loc'])
##    popuptext=r['Farbgruppe']
##    folium.Marker(location=[lat, lon],
##                  popup=popuptext#,
##                  #icon=folium.Icon(color=popupcolor, icon=popupicon) 
##                  ).add_to(m)#
#
##m.add_child(folium.LatLngPopup())  #==> Anzeigen beliebiger Koordinaten auf der Karte              
##m.add_child(folium.ClickForMarker(popup="Waypoint")) #==> zusätzl. marker hinzufügen
folium.LayerControl().add_to(m)   

#LocateControl().add_to(m)
m